In [1]:
import csv
import re
import time
import datetime
from datetime import datetime
from workalendar.asia import Taiwan
from opencc import OpenCC
import os
import dateparser
import pandas as pd
import numpy as np
import json

## Setting Rules

### translate simplified chinese into traditional chinese

def translate(source_file, result_file):
    cc = OpenCC('s2t')
    source = open(source_file, 'r', encoding = 'utf-8')
    result = open(result_file, 'w', encoding = 'utf-8')
    # source放純文字檔，轉完放result
    count = 0
    while True:
        line = source.readline()
        line = cc.convert(line)
        if not line:  #readline會一直讀下去
            break
        print(line)
        count = count +1
        result.write(line) 
        print('===已處理'+str(count)+'行===')
    source.close()        
    result.close()

for root, dirs, files in os.walk("chinese"):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)
            translate(file_path, os.path.join("chinese_traditional", file))       

---

## find target using resources from HeidelTime

repattern_lst, rules_lst, normalization_lst = [], [], []
for root, dirs, files in os.walk("chinese_traditional_edited"):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)
            txt = open(file_path, 'r', encoding = 'utf-8').read()
            if "repattern" in file:
                repattern_lst.append({"filename": file, "content": txt})
            elif "rules" in file:
                rules_lst.append({"filename": file, "content": txt})
            elif "normalization" in file:
                normalization_lst.append({"filename": file, "content": txt})

repattern_dic = []

for n in repattern_lst:
    tag = re.match(r"(re.*|tense.*)\.txt", n["filename"].split("_")[-1]).group(1)
    content_lst = n["content"].split("\n")
    repattern = [content for content in content_lst if "//" not in content and content != ""]
    
    repattern_dic.append({"tag": tag, "repattern": repattern})
    
repattern_dic

with open("repattern_dic.json", "w", encoding="utf-8") as f:
    json.dump(repattern_dic, f, indent=2, ensure_ascii=False)

rules_dic = []

for n in rules_lst:
    lines = n["content"].split("\n")
    lines = [line.replace("// ", "") for line in lines if "RULENAME" in line]

    for line in lines[1:]:
        rules_params = line.split(",")
        try:
            param_dic = {}
            for param in rules_params:
                tag, rule = param.split("=")
                param_dic[tag] = rule.replace('"', '').replace('“', '').replace('”', '')
                rules_dic.append(param_dic)
        except:
            print("---")
            print(param)
            print(n["filename"])
            
rules_dic

def match_EXTRACTION(extraction):
    for n in repattern_dic:
        if n["tag"] in extraction:
            #print("---")
            #print(n["tag"])
            #print(n["repattern"])
            regex = '|'.join(n["repattern"])
            extraction = extraction.replace("%"+n["tag"], "("+regex+")")
    return extraction

matched_rules_dic = []
for i, rule in enumerate(rules_dic):
    matched_EXTRACTION = match_EXTRACTION(rule["EXTRACTION"])
    rule_dic = rules_dic[i]
    rule_dic["matched_EXTRACTION"] = matched_EXTRACTION
    matched_rules_dic.append(rule_dic)

matched_rules_dic

with open("matched_rules_dic.json", "w", encoding="utf-8") as f:
    json.dump(matched_rules_dic, f, indent=2, ensure_ascii=False)

----------

## prepare text to be evaluated
- input string: `sentences`
- ws, pos, and ner segged list: `date_lst`, `time_lst`
- result: `results`

In [2]:
sentences = "「終於迎來這個期待已久的日子，幾天來的測試及檢驗，我再度確信A321neo是星宇航空首發機隊的正確選擇。」言談中掩飾不了內心的驕傲與期待，星宇航空董事長張國煒上周親自駕駛首架A321neo新機，從德國漢堡起飛，今天上午11點20分抵達桃園國際機場。\n過去在長榮，張國煒可以說是波音（Boeing）粉，更曾是國內第一位擁有波音777正機師及維修證照的專業航空公司董事長。從官方釋出的接機照中，可以看出張國煒在視察飛機細節時，不時露出對於A321neo的滿意神情，他甚至還真情流露地親吻機身。\n隨著台灣國籍航空首架A321neo抵台，外界也好奇，為什麼張國煒會說，A321neo是星宇航空首發機隊的正確選擇？答案可能就藏在波音的財報數字中。\n禁飛令持續，波音獲利腰斬\n波音737 MAX曾是世上最受歡迎的單走道客機，印尼獅子航空（Lion Air）、衣索比亞航空（Ethiopian Airlines）相繼發生事故，導致346人不幸喪生，這款機型從今年三月起全球停飛，如今邁入第八個月。\n這對波音公司來說，毫無疑問是一件非常燒錢的事情。根據最新財報，因為停飛，第三季737 MAX僅交付5架，遠低於去年同期的138架。\n最新一季的營運表現，營收199.80億美元，年減20%，直接反映飛機交付量減少；獲利11.67億美元，年減超過50%，其中商用飛機業務營收，更是慘跌41%至82.5億美元，自由現金流也從去年的41億美元，變成現在「負28.9億美元」，波音管理階層面臨沉重壓力，執行長米倫伯格（Dennis Muilenburg）、商用飛機最高主管麥克亞力斯特（Kevin McAllister）日前接連遭到撤換。\n對航空公司來說，飛機只要沒有在天上飛，就是在燒錢，因此大量使用737 MAX機型的航空公司，也因為禁飛令，連帶受到嚴重牽連。美國西南航空迄今因為取消超過3萬個航班，造成飛行員損失1億美元薪資。\nK董決策下得早，早早租下10架A321Neo客機反觀位處歐洲的空中巴士（Airbus），因為737 MAX的禁飛，讓旗下窄體客機大受歡迎，光是第二季的獲利就衝上12.86億美元，年成長超過4倍。\n這就可以理解，為什麼張國煒會說：「A321neo是星宇航空首發機隊的正確選擇」，現在空巴生意超好，現在改向空巴下訂單的航空公司，未必能在短期交機。張國煒在月初受訪時也說過，A321neo目前生產線滿載，平均每架飛機的交機時間都有5~6個月的延遲，「但是星宇爭取到保障名額，目前新機交機時間皆不受影響。」\n這一切都是因為張國煒決策下得早，早已簽約租下10架A321Neo客機，另外在去年七月，就跟空中巴士簽訂購機意向書（MOU），買下17架A350XWB廣體客機，完成機隊布局。\n星宇航空引進的10架A321neo是向GECAS租機公司承租，明年1月開航前將再引進2架，首航共以3架全新A321neo飛航台北至澳門、峴港及檳城。10架A321neo預定於2021年全部交付完畢。\n波音公司第2季因為737MAX的停飛，已經損失高達56億美元。瑞士信貸（Credit Suisse）分析師認為，最快要到2020年2月才有可能復飛。開航初期，星宇都將用窄體客機執飛區域航線，選擇A321neo除了能確保交機的穩定、維持營運順暢，也是對於飛航安全的宣示，張國煒當初採購飛機的決策，可以說相當神準。"

In [3]:
sentence_lst = sentences.split("\n")
sentence_lst

['「終於迎來這個期待已久的日子，幾天來的測試及檢驗，我再度確信A321neo是星宇航空首發機隊的正確選擇。」言談中掩飾不了內心的驕傲與期待，星宇航空董事長張國煒上周親自駕駛首架A321neo新機，從德國漢堡起飛，今天上午11點20分抵達桃園國際機場。',
 '過去在長榮，張國煒可以說是波音（Boeing）粉，更曾是國內第一位擁有波音777正機師及維修證照的專業航空公司董事長。從官方釋出的接機照中，可以看出張國煒在視察飛機細節時，不時露出對於A321neo的滿意神情，他甚至還真情流露地親吻機身。',
 '隨著台灣國籍航空首架A321neo抵台，外界也好奇，為什麼張國煒會說，A321neo是星宇航空首發機隊的正確選擇？答案可能就藏在波音的財報數字中。',
 '禁飛令持續，波音獲利腰斬',
 '波音737 MAX曾是世上最受歡迎的單走道客機，印尼獅子航空（Lion Air）、衣索比亞航空（Ethiopian Airlines）相繼發生事故，導致346人不幸喪生，這款機型從今年三月起全球停飛，如今邁入第八個月。',
 '這對波音公司來說，毫無疑問是一件非常燒錢的事情。根據最新財報，因為停飛，第三季737 MAX僅交付5架，遠低於去年同期的138架。',
 '最新一季的營運表現，營收199.80億美元，年減20%，直接反映飛機交付量減少；獲利11.67億美元，年減超過50%，其中商用飛機業務營收，更是慘跌41%至82.5億美元，自由現金流也從去年的41億美元，變成現在「負28.9億美元」，波音管理階層面臨沉重壓力，執行長米倫伯格（Dennis Muilenburg）、商用飛機最高主管麥克亞力斯特（Kevin McAllister）日前接連遭到撤換。',
 '對航空公司來說，飛機只要沒有在天上飛，就是在燒錢，因此大量使用737 MAX機型的航空公司，也因為禁飛令，連帶受到嚴重牽連。美國西南航空迄今因為取消超過3萬個航班，造成飛行員損失1億美元薪資。',
 'K董決策下得早，早早租下10架A321Neo客機反觀位處歐洲的空中巴士（Airbus），因為737 MAX的禁飛，讓旗下窄體客機大受歡迎，光是第二季的獲利就衝上12.86億美元，年成長超過4倍。',
 '這就可以理解，為什麼張國煒會說：「A321neo是星宇航空首發機隊的正確選擇」，現在空巴生意超好，現在改向空巴下訂單的航

In [4]:
date_lst = ['上周', '第八個月', '今年三月', '去年七月', '明年1月', '2021年', '2020年2月']
time_lst = ['上午11點20分']

In [5]:
results = []

In [6]:
results = results + [{"source": n, "target": ""} for n in date_lst] + [{"source": n, "target": ""} for n in time_lst]
results

[{'source': '上周', 'target': ''},
 {'source': '第八個月', 'target': ''},
 {'source': '今年三月', 'target': ''},
 {'source': '去年七月', 'target': ''},
 {'source': '明年1月', 'target': ''},
 {'source': '2021年', 'target': ''},
 {'source': '2020年2月', 'target': ''},
 {'source': '上午11點20分', 'target': ''}]

---

## find target using the dateparse package

In [7]:
for n in date_lst:
    try:
        target = dateparser.parse(n)
        if not target == None:
            results.append({"source": n, "target": target})
    except:
        pass

for n in time_lst:
    try:
        target = dateparser.parse(n)
        if not target == None:
            results.append({"source": n, "target": target})
    except:
        pass
    
results

[{'source': '上周', 'target': ''},
 {'source': '第八個月', 'target': ''},
 {'source': '今年三月', 'target': ''},
 {'source': '去年七月', 'target': ''},
 {'source': '明年1月', 'target': ''},
 {'source': '2021年', 'target': ''},
 {'source': '2020年2月', 'target': ''},
 {'source': '上午11點20分', 'target': ''},
 {'source': '上周',
  'target': datetime.datetime(2019, 11, 19, 16, 22, 9, 180406)},
 {'source': '2020年2月', 'target': datetime.datetime(2020, 2, 26, 0, 0)}]

## find target using resources from HeidelTime

In [8]:
with open("matched_rules_dic.json", "r", encoding="utf-8") as f:
    matched_rules_dic = json.load(f)

In [10]:
def HeidelTime_detect(string):
    found = []
    for i, rule in enumerate(matched_rules_dic):
        try:
            attempt = re.search(rule["matched_EXTRACTION"], string)
            if attempt.group(0) in found:
                pass
            else:
                found.append(attempt.group(0))
        except:
            pass
    return found

In [11]:
for sentence in sentence_lst:
    sources = HeidelTime_detect(sentence)
    if sources != []:
        for source in sources:
            results.append({"source": source, "target": ""})

results

[{'source': '上周', 'target': ''},
 {'source': '第八個月', 'target': ''},
 {'source': '今年三月', 'target': ''},
 {'source': '去年七月', 'target': ''},
 {'source': '明年1月', 'target': ''},
 {'source': '2021年', 'target': ''},
 {'source': '2020年2月', 'target': ''},
 {'source': '上午11點20分', 'target': ''},
 {'source': '上周',
  'target': datetime.datetime(2019, 11, 19, 16, 22, 9, 180406)},
 {'source': '2020年2月', 'target': datetime.datetime(2020, 2, 26, 0, 0)},
 {'source': '今天', 'target': ''},
 {'source': '今天上午', 'target': ''},
 {'source': '上周', 'target': ''},
 {'source': '上午11點20分', 'target': ''},
 {'source': '過去', 'target': ''},
 {'source': '今年三月', 'target': ''},
 {'source': '三月', 'target': ''},
 {'source': '如今', 'target': ''},
 {'source': '今年', 'target': ''},
 {'source': '八個月', 'target': ''},
 {'source': '第三季', 'target': ''},
 {'source': '去年同期', 'target': ''},
 {'source': '三季', 'target': ''},
 {'source': '現', 'target': ''},
 {'source': '新一季', 'target': ''},
 {'source': '一季', 'target': ''},
 {'source': '第二季'

In [12]:
[HeidelTime_detect(n) for n in date_lst]

[['上周'],
 ['八個月'],
 ['今年三月', '三月', '今年'],
 ['去年七月', '七月', '去年'],
 ['明年1月', '1月', '明年'],
 ['2021年'],
 ['2020年2月', '2020年', '2月']]

In [13]:
[HeidelTime_detect(n) for n in time_lst]

[['上午11點20分']]

In [14]:
df = pd.DataFrame(results)
df = df.replace("", np.nan, regex=True)
df = df.groupby("source").first()
df = df.sort_values("source").reset_index()
df.index = df.index + 1
df

,source,target
1,1月,NaN
2,2020年,NaN
3,2020年2月,2020-02-26 00:00:00
4,2021年,NaN
5,2月,NaN
6,一季,NaN
7,七月,NaN
8,三季,NaN
9,三月,NaN
10,上午11點20分,NaN


---

## find holidays using the workalendar package and holiday_2020 csv file

with open("holidays_2020.csv", "r", encoding="utf-8") as f:
    rows = csv.reader(f)
    
    holidays_2020 = []
    for row in rows:
        if row[3] != "": # no holidays in the csv file
            d = re.match("(\d+)年(\d+)月(\d+)日", row[0])
            padded_date = "{}年{}月{}日".format(d.group(1), d.group(2).zfill(2), d.group(3).zfill(2))
            holidays_2020.append((datetime.strptime(padded_date, "%Y年%m月%d日").date(), row[3]))

def working_day(d):
    cal = Taiwan()
    return cal.is_working_day(d)

def holiday(d):
    holiday = False
    for k, v in holidays_2020:
        if d == k:
            holiday = True
    return holiday